In [1]:
import tensorflow as tf
import numpy as np

import pickle

def load_pkl(file_path) :
    
    with open(file_path, 'rb') as f:
        df = pickle.load(f)
        
    return df

In [2]:
def create_training_instances(input_files,
                              tokenizer,
                              max_seq_length,
                              dupe_factor,
                              short_seq_prob,
                              masked_lm_prob,
                              max_predictions_per_seq,
                              rng,
                              do_whole_word_mask=False,
                              max_ngram_size=None) :
    
    all_documents = [[]]
    
    for input_file in input_files :
        
        inputs = load_pkl(input_file)
        keys = list(inputs.keys())
        
        for key in keys :
            tokens = [ tokenizer(x)['input_ids'] for x in inputs[key] if x ]

            all_documents.append(tokens)
        
    
    all_documents = [x for x in all_documents if x]
    rng.shuffle(all_documents)
    
#     vocab_words = list(tokenizer.vocab.keys())
#     instances = []
    
#     for _ in range(dupe_factor):
#         for document_index in range(len(all_documents)):
#             instances.extend(
#                 create_instances_from_document(
#                       all_documents, document_index, max_seq_length, short_seq_prob,
#                       masked_lm_prob, max_predictions_per_seq, vocab_words, rng,
#                       do_whole_word_mask, max_ngram_size))

#     rng.shuffle(instances)
    return all_documents
    
        

In [8]:
tokenizer_for_load.

AttributeError: 'BertTokenizerFast' object has no attribute 'vocab'

In [3]:

from transformers import BertTokenizerFast
import random

file_path = ['../dt/wiki_ko_dict_sample.pkl']
tokenizer_for_load = BertTokenizerFast.from_pretrained('../model/BertTokenizer-3000-32000-vocab.txt'
                                                   , strip_accents=False
                                                   , lowercase=False)
ans = create_training_instances(input_files=file_path
                              , tokenizer = tokenizer_for_load
                              , max_seq_length = 130
                              , dupe_factor = 2
                              , short_seq_prob = .15
                              , masked_lm_prob = .15
                              , max_predictions_per_seq = 255
                              , rng = random.Random(123)
                              , do_whole_word_mask=False)

Calling BertTokenizerFast.from_pretrained() with the path to a single file or url is deprecated


In [ ]:
tokenizer_for_load.

In [24]:
tokenizer_for_load("한규인")

{'input_ids': [2, 2817, 3625, 3105, 3], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1]}

In [16]:
tokenizer_for_load.tokenize('한규인은 과연 무엇을', "어떻게")

['한', '##규', '##인은', '과연', '무엇을', '어떻게']

In [ ]:
def create_instances_from_document(
    all_documents, document_index, max_seq_length, short_seq_prob,
    masked_lm_prob, max_predictions_per_seq, vocab_words, rng,
    do_whole_word_mask=False, max_ngram_size=None):